# Main Function

Need to do the following in terminal to run:
mkdir tmp
mkdir tmp/ppo
mkdir plots


In [4]:
import gym
import torch
import numpy as np
from ipynb.fs.full.CartPolePPO import Agent
from utils import plot_learning_curve

In [5]:
if __name__ == '__main__':
    env = gym.make('CartPole-v1')
    N = 20
    batch_size = 5
    n_epochs = 4
    alpha = 0.0003

    agent = Agent(num_actions = env.action_space.n, batch_size = 5, alpha = 0.0003, num_epochs = 4, input_dims = env.observation_space.shape)
    n_games = 400

    figure_file = 'plots/CartPole.png'

    best_score = env.reward_range[0]
    score_history = []

    learn_iters = 0
    avg_score = 0
    n_steps = 0
    steps = 20

    agent.load_models()


    for i in range(n_games): # 103 min 8.8 sec for 300 iterations
        observation = env.reset()
        # observation = prepro(observation)
        done = False
        score = 0
        while not done:
            action, prob, val = agent.choose_action(observation)
            observation_, reward, done, info = env.step(action)
            n_steps += 1
            score += reward
            agent.remember(observation, action, prob, val, reward, done)
            if n_steps % N == 0: # if true, it's time to perform learning function
                agent.learn()
                learn_iters += 1
            observation = observation_
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])

        if avg_score > best_score and i > steps: # if best score found
            best_score = avg_score
            agent.save_models()

        print('episode', i, 'score %.1f' % score, 'avg score %.1f' % avg_score,
                'time_steps', n_steps, 'learning_steps', learn_iters)
    x = [i+1 for i in range(len(score_history))]
    plot_learning_curve(x, score_history, figure_file, "Training Episodes", "Average Scores", "Pong")


... loading models ...
episode 0 score 500.0 avg score 500.0 time_steps 500 learning_steps 25
episode 1 score 385.0 avg score 442.5 time_steps 885 learning_steps 44
episode 2 score 500.0 avg score 461.7 time_steps 1385 learning_steps 69
episode 3 score 500.0 avg score 471.2 time_steps 1885 learning_steps 94
episode 4 score 500.0 avg score 477.0 time_steps 2385 learning_steps 119
episode 5 score 29.0 avg score 402.3 time_steps 2414 learning_steps 120
episode 6 score 500.0 avg score 416.3 time_steps 2914 learning_steps 145
episode 7 score 315.0 avg score 403.6 time_steps 3229 learning_steps 161
episode 8 score 302.0 avg score 392.3 time_steps 3531 learning_steps 176
episode 9 score 143.0 avg score 367.4 time_steps 3674 learning_steps 183
episode 10 score 22.0 avg score 336.0 time_steps 3696 learning_steps 184
episode 11 score 102.0 avg score 316.5 time_steps 3798 learning_steps 189
episode 12 score 18.0 avg score 293.5 time_steps 3816 learning_steps 190
episode 13 score 16.0 avg score 27

TypeError: plot_learning_curve() missing 1 required positional argument: 'title'

# Save Trained Model

In [4]:
# agent.save_models()

... saving models ...


# Load Trained Model and Run Iteration

In [ ]:
# Load model
agent.load_models()

In [4]:
env = gym.make('CartPole-v1')
env.reset()
score = 0
done = False
n_steps = 0
while not done:
    env.render()
    action, prob, val = agent.choose_action(observation)
    observation_, reward, done, info = env.step(action)
    n_steps += 1
    score += reward
    agent.remember(observation, action, prob, val, reward, done)
    if n_steps % N == 0: # if true, it's time to perform learning function
        agent.learn()
        learn_iters += 1
    observation = observation_
print(score)
env.close()


79.0
